In [1]:
from pyspark.sql import SparkSession
# Initialize SparkSession
spark = SparkSession.builder \
    .appName("Bank Transactions") \
    .getOrCreate()

# Load the dataset
data = [("acc123", 100), ("acc456", 200)]
columns = ["account", "balance"]
df = spark.createDataFrame(data, columns)

# Function to update the balance
def update_balance(account, amount):
    # Retrieve current balance
    current_balance = df.filter(df.account ==
account).select("balance").collect()[0][0]
    # Update balance
    new_balance = current_balance + amount
    # Simulate processing delay
    import time
    time.sleep(2)
    # Update dataframe with new balance
    df.filter(df.account == account).update({"balance": new_balance})

# Simulate concurrent transactions
from concurrent.futures import ThreadPoolExecutor

with ThreadPoolExecutor(max_workers=2) as executor:
    executor.submit(update_balance, "acc123", 50)
    executor.submit(update_balance, "acc123", -30)
# Display updated dataframe
df.show()

24/04/29 21:52:52 WARN Utils: Your hostname, bhaskara15 resolves to a loopback address: 127.0.1.1; using 172.20.252.160 instead (on interface enp3s0)
24/04/29 21:52:52 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/29 21:52:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


+-------+-------+
|account|balance|
+-------+-------+
| acc123|    100|
| acc456|    200|
+-------+-------+



In [5]:
from pyspark.sql import SparkSession

# Initialize SparkSession
spark = SparkSession.builder \
    .appName("Bank Transactions") \
    .getOrCreate()

# Load the dataset
data = [("acc123", 100), ("acc456", 200)]
columns = ["account", "balance"]
df = spark.createDataFrame(data, columns)

# Function to transfer funds between accounts
def transfer_funds(from_account, to_account, amount):
    # Retrieve current balances
    from_balance = df.filter(df.account ==
from_account).select("balance").collect()[0][0]
    to_balance = df.filter(df.account == to_account).select("balance").collect()[0][0]
    # Update balances
    new_from_balance = from_balance - amount
    new_to_balance = to_balance + amount
    # Simulate processing delay
    import time
    time.sleep(2)
    # Update dataframe with new balances
    df.filter(df.account == from_account).update({"balance": new_from_balance})
    df.filter(df.account == to_account).update({"balance": new_to_balance})

# Simulate concurrent transactions
from concurrent.futures import ThreadPoolExecutor

with ThreadPoolExecutor(max_workers=2) as executor:
    executor.submit(transfer_funds, "acc123", "acc456", 50)
    executor.submit(transfer_funds, "acc456", "acc123", 30)

# Display updated dataframe
df.show()

+-------+-------+
|account|balance|
+-------+-------+
| acc123|    100|
| acc456|    200|
+-------+-------+



In [6]:
from pyspark.sql import SparkSession
# Initialize SparkSession
spark = SparkSession.builder \
    .appName("Bank Transactions") \
    .getOrCreate()

# Load the dataset
data = [("acc123", 100), ("acc456", 200)]
columns = ["account", "balance"]
df = spark.createDataFrame(data, columns)

# Function to transfer funds between accounts
def transfer_funds(from_account, to_account, amount):
    # Retrieve current balances
    from_balance = df.filter(df.account ==
from_account).select("balance").collect()[0][0]
    to_balance = df.filter(df.account == to_account).select("balance").collect()[0][0]
    # Update balances
    new_from_balance = from_balance - amount
    new_to_balance = to_balance + amount
    # Simulate processing delay
    import time
    time.sleep(2) # Introduce delay after updating the balance in the first account
    # Update dataframe with new balances
    df.filter(df.account == from_account).update({"balance": new_from_balance})
    # Introduce an artificial failure here to simulate a failure after updatingthe first account
    # This failure will simulate the scenario where the transaction fails before updating the second account
    # For example, a system crash or network failure
    raise Exception("Simulated failure before updating the second account")

# Simulate a single transaction
try:
    transfer_funds("acc123", "acc456", 50)
except Exception as e:
    print("Exception occurred:", str(e))
    
# Display updated dataframe (to show the inconsistent state)
df.show()

Exception occurred: 'DataFrame' object has no attribute 'update'
+-------+-------+
|account|balance|
+-------+-------+
| acc123|    100|
| acc456|    200|
+-------+-------+

